# Recipe Generator Using Pretrained AI
This notebook generates recipes based on user-provided ingredients using a pretrained AI model (e.g., GPT-4).

In [1]:
import torch
import gc
import os
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

/Users/edrikisaian/Documents/Advanced_AI/AdvancedAI_IndividualProject/venv311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'

In [3]:
def clear_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    if torch.backends.mps.is_available():
        torch.mps.empty_cache()
    gc.collect()


In [4]:
if torch.backends.mps.is_available():
    device = "mps"
    print("Using Metal (MPS) for inference")
else:
    device = "cpu"
    print("Using CPU for inference")


Using Metal (MPS) for inference


In [5]:
def load_model():
    model_name = "EleutherAI/gpt-neo-1.3B"
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPTNeoForCausalLM.from_pretrained(model_name)
    model.to(device)
    return model, tokenizer


In [6]:
def get_users_ingredients():
    users_ingredients = []
    print("Enter the ingredients you have (type 'end' to finish):")
    while True:
        user_input = input("Ingredient: ").strip().lower()
        if user_input == 'end':
            break
        if user_input:
            users_ingredients.append(user_input)
    return users_ingredients


In [7]:
def generate_recipe(ingredients, model, tokenizer, max_loops=5):
    prompt = (
        f"Given these ingredients: {', '.join(ingredients)}, suggest 1–3 recipes I can make. "
        "For each recipe, provide a short and structured list of steps. Use bullet points. "
        "Keep the instructions brief, clear, and limited to essential steps only. Do not add extra explanation or repetition."
    )

    generated_text = prompt
    for i in range(max_loops):
        inputs = tokenizer(generated_text, return_tensors="pt", truncation=True, max_length=1024)
        inputs = {key: value.to(device) for key, value in inputs.items()}

        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.7,
            top_k=50,
            top_p=0.95,
            no_repeat_ngram_size=3,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

        output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        if output_text == generated_text:
            break
        generated_text = output_text
        if "enjoy your meal" in output_text.lower() or "bon appétit" in output_text.lower():
            break

    return generated_text

In [8]:
def main():
    clear_memory()

    print("Welcome to the AI Recipe Generator!")

    model, tokenizer = load_model()

    ingredients = get_users_ingredients()
    if not ingredients:
        print("No ingredients provided. Exiting.")
        return

    print("\nGenerating recipe suggestions...\n")
    try:
        recipe = generate_recipe(ingredients, model, tokenizer)
        print("Here are the recipe suggestions:\n")
        print(recipe)
    except Exception as e:
        print("Error generating recipe:\n")
        print(e)
        print("\nSorry, I couldn't generate a recipe. Please try again.")
    
    clear_memory()

In [9]:
main()

Welcome to the AI Recipe Generator!
Enter the ingredients you have (type 'end' to finish):

Generating recipe suggestions...

Here are the recipe suggestions:

Given these ingredients: bread, eggs, flour, bread, chicken, flour, suggest 1–3 recipes I can make. For each recipe, provide a short and structured list of steps. Use bullet points. Keep the instructions brief, clear, and limited to essential steps only. Do not add extra explanation or repetition.

_The Bread_

**1.** Bread dough

1.1. Bread dough preparation

Pour boiling water over the dry ingredients.
2. Bread flour

2-1. Place flour in a large bowl.
3. Prepare dough.
4. Bread

4-1-1 Begin by preparing the dough. Place the flour in the bowl. Place a quarter of the flour on top of the dry mixture.
5. Cut the dough in half.
6. Cut each half into 8 pieces.
7. Place pieces on the bowl with the remaining flour.
8. Roll the pieces into balls.
9. Put the balls on the flour-covered surface.
10. Roll out the balls. Place them on a gre

In [10]:
# torch.mps.empty_cache()
# gc.collect()
